In [ ]:
def create_random_binary_classifier(category):
    """
    Crea un classificatore binario casuale che predice solo la categoria specificata.

    :param category: La categoria che il classificatore predirà
    :return: Una funzione che genera previsioni casuali della categoria specificata
    """
    def random_binary_classifier(size):
        """
        Genera previsioni casuali della categoria specificata.

        :param size: Numero di previsioni da generare
        :return: Array di previsioni casuali della categoria specificata
        """
        return np.random.choice([0, 1], size=size)
    
    return random_binary_classifier

In [ ]:
for category in categories:
    globals()[f'random_binary_classifier_{category}'] = create_random_binary_classifier(category)

In [ ]:
for category in categories:
    classifier = globals()[f'random_binary_classifier_{category}']
    df_test[f'predicted_{category}'] = classifier(len(df_test))

In [ ]:
print(globals()[f'random_binary_classifier_Openess to change'])

In [ ]:
#define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('premise', CountVectorizer(), 'Premise'),
        ('conclusion', CountVectorizer(), 'Conclusion'),
        ('stance', OneHotEncoder(), 'Stance')
    ])

# Applicare il ColumnTransformer al DataFrame
df_transformed = preprocessor.fit_transform(df_test)

In [ ]:
#apply the BERT w_C classifiers to the test set
for classifier_name, classifier in classifiers.items():
    predictions = classifier(df_test['Conclusion'].tolist())
    df_test[f'predicted_{classifier_name}'] = predictions

print(df_test)

#apply the BERT w_CPS classifiers to the test set WARNING: QUI SERVE DI CONVERTIRE IN LISTA ANCHE STANCE?
for classifier_name, classifier in classifiers.items():
    predictions = classifier(df_test['Conclusion'].tolist(), df_test['Premise'].tolist(), df_test['Stance'].tolist())
    df_test[f'predicted_{classifier_name}'] = predictions

In [ ]:
# Example usage
categories = ['category1', 'category2']  # Replace with actual categories
y_true = {
    'category': df_test['category'].tolist(),
    'Stance': df_test['Stance_encoded'].tolist()
}
y_pred = [...]  # Replace with predicted labels

# Calculate per-category F1 scores
category_f1_scores = calculate_per_category_f1(y_true, y_pred, categories)

# Calculate average F1 score
average_f1 = calculate_average_f1(category_f1_scores)

print("Per-category F1 scores:", category_f1_scores)
print("Average F1 score:", average_f1)

In [ ]:
    train_dataset = TextDataset(train_texts, train_labels, tokenizer, max_len=128)
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

    # Train the model
    train_bert_classifier(model, train_dataloader)

    # Save the trained model in the dictionary
    bert_name = f'BERT_w/CPS_{category}'
    classifiers[bert_name] = model

In [ ]:
# Training function
def train_bert_classifier(model, dataloader, epochs=3, lr=2e-5):
    optimizer = AdamW(model.parameters(), lr=lr)
    model.train()

    for epoch in range(epochs):
        for batch in dataloader:
            optimizer.zero_grad()
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['label']
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

In [ ]:

    # Create dataset and dataloader for validation
    val_dataset = TextDataset(val_texts, val_labels, tokenizer, max_len=128)
    val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

    # Evaluate the model
    model.eval()
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['label']
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    # Calculate per-category F1 scores
    y_true = {
        'category': [category] * len(val_labels),
        'Stance': val_labels
    }
    category_f1_scores = calculate_per_category_f1(y_true, val_preds, [category])

    # Print per-category F1 scores
    print(f"Per-category F1 scores for {category}:", category_f1_scores)